# Milestone 3: LSTM Model Development and Evaluation

## Objective
Develop an LSTM model to predict energy consumption using a sliding window approach. Evaluate the model against the baseline Linear Regression model and prepare the best model for deployment.

## Steps
1. **Data Loading & Preparation**: Load preprocessed data, scale, and create sequences.
2. **LSTM Model Development**: Build, train, and save the LSTM model.
3. **Evaluation**: Compare LSTM with Baseline Linear Regression.
4. **Prediction Function**: Create a function for Flask integration.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import pickle
import os

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("Libraries imported successfully.")

## 1. Data Loading & Preparation

In [ ]:
# Load the preprocessed data
data_path = '../milestone2/feature_engineered_data.csv'
df = pd.read_csv(data_path)

# Convert timestamp to datetime if not already
# Assuming 'Datetime' is the column name, adjust if necessary after inspection
if 'Datetime' in df.columns:
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df.set_index('Datetime', inplace=True)

print("Data loaded. Shape:", df.shape)
df.head()

In [ ]:
# Define the target variable
target_col = 'Global_active_power'  # Adjust if column name is different

# Ensure the data is sorted by time
df.sort_index(inplace=True)

# Use only the target column for sequence generation (univariate time series for now)
# If multivariate is needed, include other features here
data = df[[target_col]].values

# Scaling
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

print("Data scaled.")

In [ ]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

SEQ_LENGTH = 24  # Past 24 hours

X, y = create_sequences(scaled_data, SEQ_LENGTH)

print("Sequences created.")
print("X shape:", X.shape)
print("y shape:", y.shape)

In [ ]:
# Split into Train (70%), Validation (15%), Test (15%)
train_size = int(len(X) * 0.70)
val_size = int(len(X) * 0.15)
test_size = len(X) - train_size - val_size

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

print("Train shape:", X_train.shape, y_train.shape)
print("Val shape:", X_val.shape, y_val.shape)
print("Test shape:", X_test.shape, y_test.shape)

## 2. LSTM Model Development

In [ ]:
def build_lstm_model(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(32, return_sequences=False),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

input_shape = (X_train.shape[1], X_train.shape[2])
model = build_lstm_model(input_shape)
model.summary()

In [ ]:
# Training configuration
epochs_list = [20, 40, 60]
batch_sizes = [16, 32]
best_val_loss = float('inf')
best_model = None
best_history = None
best_params = {}

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Grid search for epochs and batch size (simplified)
# Note: In a real scenario, we might want to separate this loop or use a proper tuner.
# For this task, we will train with one config and then maybe mention others or loop.
# Let's stick to the prompt: "Train model using Epochs: test with 20, 40, and 60. Batch sizes: 16 and 32."
# To save time, I will iterate but update best model.

for epochs in epochs_list:
    for batch_size in batch_sizes:
        print(f"Training with params: epochs={epochs}, batch_size={batch_size}")
        
        # Re-build model to start fresh
        temp_model = build_lstm_model(input_shape)
        
        history = temp_model.fit(
            X_train, y_train,
            epochs=epochs,
            batch_size=batch_size,
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            verbose=0  # Reduce output
        )
        
        val_loss = min(history.history['val_loss'])
        print(f"Best Val Loss: {val_loss}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = temp_model
            best_history = history
            best_params = {'epochs': epochs, 'batch_size': batch_size}

print("Best Params:", best_params)
print("Best Val Loss:", best_val_loss)

In [ ]:
# Plot training vs validation loss for the best model
plt.figure(figsize=(10, 6))
plt.plot(best_history.history['loss'], label='Train Loss')
plt.plot(best_history.history['val_loss'], label='Validation Loss')
plt.title(f'Model Loss (Epochs={best_params["epochs"]}, Batch={best_params["batch_size"]})')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.show()

In [ ]:
# Save the best LSTM model
best_model.save('lstm_energy_model.h5')
print("LSTM model saved as 'lstm_energy_model.h5'.")

## 3. Evaluation & Comparison

In [ ]:
# Evaluate LSTM on Test Set
lstm_preds_scaled = best_model.predict(X_test)
lstm_preds = scaler.inverse_transform(lstm_preds_scaled)
y_test_actual = scaler.inverse_transform(y_test)

lstm_mae = mean_absolute_error(y_test_actual, lstm_preds)
lstm_rmse = np.sqrt(mean_squared_error(y_test_actual, lstm_preds))
lstm_r2 = r2_score(y_test_actual, lstm_preds)

print("LSTM Results:")
print(f"MAE: {lstm_mae}")
print(f"RMSE: {lstm_rmse}")
print(f"R2: {lstm_r2}")

In [ ]:
# Load Baseline Model
baseline_model_path = '../milestone2/baseline_linear_regression.pkl'
with open(baseline_model_path, 'rb') as f:
    baseline_model = pickle.load(f)
    
# Check what the baseline model expects as input.
# Typically, Linear Regression might expect features like [Lag_1, Lag_2, ...]. 
# We need to reconstruct the test set for the baseline model to be comparable.
# Assuming the baseline used similar features or we use the 'feature_engineered_data.csv' directly.
# Since the request says "Evaluate both", I'll assume we can use the same X_test if it was shaped correctly, 
# OR we need to use the original dataframe's test split features.

# Let's check the columns of the original df to see what features were used.
print(df.columns)

# Attempt to load X_test and y_test consistent with Milestone 2 if possible.
# For now, as a placeholder, I will assume the baseline model can predict on the same features 
# OR I will assume we have to create the features again.

# IMPORTANT: The LSTM uses just the 'Global_active_power' sequence.
# The Linear Regression likely used 'Lag_1', 'Lag_2', 'Rolling_Mean', etc.
# We need to prepare data for Baseline.
# I'll try to find common indices in X_test (which corresponds to timestamps) and extract features for those.

# Reconstruct timestamps for X_test
# Timestamps for y_test are from (train_size + val_size + seq_length) to end
test_start_idx = train_size + val_size + SEQ_LENGTH
test_timestamps = df.index[test_start_idx : test_start_idx + len(y_test)]

# Extract features for these timestamps. 
# We need to drop 'Global_active_power' and 'Datetime' if they are in features.
feature_cols = [c for c in df.columns if c not in ['Global_active_power', 'Datetime']]
X_test_baseline = df.loc[test_timestamps, feature_cols]
y_test_baseline = df.loc[test_timestamps, 'Global_active_power']

# Handle missing features if any (though preprocessing should have handled it)
X_test_baseline.fillna(0, inplace=True)

try:
    baseline_preds = baseline_model.predict(X_test_baseline)
    
    baseline_mae = mean_absolute_error(y_test_baseline, baseline_preds)
    baseline_rmse = np.sqrt(mean_squared_error(y_test_baseline, baseline_preds))
    baseline_r2 = r2_score(y_test_baseline, baseline_preds)
    
    print("Baseline Results:")
    print(f"MAE: {baseline_mae}")
    print(f"RMSE: {baseline_rmse}")
    print(f"R2: {baseline_r2}")
except Exception as e:
    print(f"Could not evaluate baseline model: {e}")
    baseline_mae, baseline_rmse, baseline_r2 = np.nan, np.nan, np.nan

In [ ]:
# Comparison Table
results = {
    'Model': ['Linear Regression', 'LSTM'],
    'MAE': [baseline_mae, lstm_mae],
    'RMSE': [baseline_rmse, lstm_rmse],
    'R2': [baseline_r2, lstm_r2]
}
results_df = pd.DataFrame(results)
print(results_df)

# Select Best Model
if lstm_rmse < baseline_rmse:
    print("Best Model: LSTM")
    best_final_model = best_model
    best_model_name = 'best_energy_model.h5'
    best_final_model.save(best_model_name)
else:
    print("Best Model: Linear Regression")
    # Baseline is already saved as pkl, we can copy it or just reference it.
    # Instruction says save as best_energy_model.pkl
    best_model_name = 'best_energy_model.pkl'
    with open(best_model_name, 'wb') as f:
        pickle.dump(baseline_model, f)

print(f"Best model saved as {best_model_name}")

## 4. Flask-Compatible Prediction Function

In [ ]:
def predict_energy(input_sequence):
    """
    input_sequence: Array-like of shape (24,) representing past 24 hours of energy data.
    returns: Predicted energy value.
    """
    # Ensure input is numpy array
    input_seq = np.array(input_sequence)
    
    # Reshape for scalar
    input_seq = input_seq.reshape(-1, 1)
    
    # Scale
    # Note: We should use the scaler fitted on training data
    scaled_seq = scaler.transform(input_seq)
    
    # Reshape for LSTM (1, 24, 1)
    model_input = scaled_seq.reshape(1, SEQ_LENGTH, 1)
    
    # Predict
    scaled_prediction = best_model.predict(model_input, verbose=0)
    
    # Inverse scale
    prediction = scaler.inverse_transform(scaled_prediction)
    
    return prediction[0][0]

# Test with a sample from test set
sample_idx = 0
sample_input = df['Global_active_power'].iloc[test_start_idx + sample_idx - SEQ_LENGTH : test_start_idx + sample_idx].values

print("Sample Input Shape:", sample_input.shape)
predicted_value = predict_energy(sample_input)
print(f"Predicted Energy: {predicted_value}")
print(f"Actual Energy: {y_test_actual[sample_idx][0]}")